### Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

In [130]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import os
import csv

csvpath = os.path.join("Resources/purchase_data.csv")
heroes_df = pd.read_csv(csvpath, encoding="UTF-8")
heroes_df.head(2)

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56


In [131]:
player_count = len(heroes_df["SN"].unique())
player_count_disp = pd.DataFrame({"Player Count": [player_count]})
player_count_disp

,Player Count
0,576


In [132]:
unique = len(heroes_df["Item ID"].unique())
avg_price = heroes_df["Price"].mean()
purchases = heroes_df["SN"].count()
revenue = heroes_df["Price"].sum()

# creating a new DF to hold the results
summary_df = pd.DataFrame({"Number of Unique Items": [unique], "Average Price": [avg_price], "Number of Purchases": 
                           [purchases], "Total Revenue": [revenue]})

summary_df["Average Price"] = summary_df["Average Price"].map("${:.2f}".format)
summary_df["Total Revenue"] = summary_df["Total Revenue"].map("${:.2f}".format)

summary_df.head()

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,$2379.77


In [133]:
# Calculating total males and females and finding the percentage 
# Drop the duplicates SN first

duplicate = heroes_df.drop_duplicates(subset='SN', keep="first")

male_count = duplicate[duplicate.Gender == 'Male'].shape[0]
female_count = duplicate[heroes_df.Gender == 'Female'].shape[0]
total_count = male_count + female_count
other_count = total_count - male_count - female_count
male_percentage = (male_count * 100) / total_count
female_percentage = (female_count * 100) / total_count
other_percentage = (other_count * 100) / total_count

# Creating a new DF
gender_percentage_disp_df = pd.DataFrame(
    {"Gender": ['Male', 'Female', 'Other/Non-Disclosed'],
     'Total Count': [male_count, female_count, other_count],
     'Percentage': [male_percentage, female_percentage, other_percentage]})
                                                                    
# Mapping the percentage
gender_percentage_disp_df['Percentage'] = gender_percentage_disp_df["Percentage"].map("{:.2f}%".format)
gender_percentage_disp_df = gender_percentage_disp_df.set_index('Gender')
gender_percentage_disp_df

# ~~~~~~~~~~~Another way to display the data
#gender_percentage_disp_df["Male Percentage"] = gender_percentage_disp_df["Male Percentage"].map("${:.2f}".format)
#gender_percentage_disp_df = pd.DataFrame({"Male":[male_count], "Female":[female_count],
                                      #"Male Percentage":[male_percentage], "Female Percentage":[female_percentage]})
#~~~~~~~~~~~~~

/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


,Total Count,Percentage
Gender,,
Male,484,85.66%
Female,81,14.34%
Other/Non-Disclosed,0,0.00%



## Purchasing Analysis (Gender)

In [138]:
# Calculating the total purchases for genders
#total_purchases_df = heroes_df.groupby["Gender"].value_counts()
#total_purchases_df.head()
total_purchases = heroes_df.groupby(["Gender"]).sum()["Price"]

#count_purchases = heroes_df.groupby["Gender"].count()

avg_purchase_price = heroes_df.groupby("Gender")["Price"].mean()
#avg_purchase_price_df

total_value = total_purchases * avg_purchase_price
#total_value_df
#avg_per_person_df = heroes_df.groupby("Gender")

#purchase_count = heroes_df["SN"].count()
#purchase_count

        
# Creating a new DF
purchase_df = pd.DataFrame(
    {"Purchase Count": total_purchases,
     "Average Purchase Price": avg_purchase_price,
    "Total Purchase Amount": total_value})

purchase_df["Average Purchase Price"] = purchase_df["Average Purchase Price"].map("${:.2f}".format)
purchase_df["Total Purchase Amount"] = purchase_df["Total Purchase Amount"].map("${:.2f}".format)
            
purchase_df


,Purchase Count,Average Purchase Price,Total Purchase Amount
Gender,,,
Female,361.94,$3.20,$1159.30
Male,1967.64,$3.02,$5938.05
Other / Non-Disclosed,50.19,$3.35,$167.94


## Age Demographics

In [135]:
num_bins = 8
# Creating bins and labels
bins = [0,10,15,20,25,30,35,40,200]
labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"] 

#making a copy of the existing DF
ages_bins = heroes_df.copy()
ages_bins = heroes_df.drop_duplicates(subset='SN', keep="first")

# cutting the DF by Age
ages_bins["Age Group"] = pd.cut(ages_bins["Age"], bins, labels=labels)

# Grouping by Ages
ages_bins = ages_bins.groupby(["Age Group"])

total_count = ages_bins["SN"].count()

percentage = (total_count * 100) / total_count.sum()

new_ages_bin_df = pd.DataFrame({"Total Count": total_count, "Percentage": percentage})
new_ages_bin_df["Percentage"] = new_ages_bin_df["Percentage"].map("{:.2f}%".format)
new_ages_bin_df.head(10)   


#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~Total count is different 

/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


,Total Count,Percentage
Age Group,,
<10,24,4.17%
10-14,41,7.12%
15-19,150,26.04%
20-24,232,40.28%
25-29,59,10.24%
30-34,37,6.42%
35-39,26,4.51%
40+,7,1.22%


## Purchasing Analysis (Age)

In [146]:
# purchase count

num_bins = 8
# Creating bins and labels
bins = [0,10,15,20,25,30,35,40,200]
labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"] 

#making a copy of the existing DF
new_bins = heroes_df.copy()
new_bins = heroes_df.drop_duplicates(subset='SN', keep="first")

# cutting the DF by Age
new_bins["Age Group"] = pd.cut(new_bins["Age"], bins, labels=labels)

# Grouping by Ages
new_bins = new_bins.groupby(["Age Group"])

# Simple calculations 
purchases = new_bins["SN"].count()
revenue = new_bins["Price"].sum()
avg_price = revenue / purchases
avg_total_per_person = new_bins["Price"].sum() / new_bins["SN"].count()

# Creating a new DF
analysis_df = pd.DataFrame({"Purchase Count": purchases, 
                        "Average Purchase Price": avg_price, 
                        "Total Purchase Value": revenue,
                       "Average Total Purchase Per Person": avg_total_per_person})

# Cleaning the format
analysis_df["Average Purchase Price"] = analysis_df["Average Purchase Price"].map("${:.2f}".format)
analysis_df["Total Purchase Value"] = analysis_df["Total Purchase Value"].map("${:.2f}".format)
analysis_df["Average Total Purchase Per Person"] = analysis_df["Average Total Purchase Per Person"].map("${:.2f}".format)

analysis_df


/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase Per Person
Age Group,,,,
<10,24,$3.42,$82.18,$3.42
10-14,41,$2.94,$120.43,$2.94
15-19,150,$3.18,$476.84,$3.18
20-24,232,$3.02,$700.03,$3.02
25-29,59,$2.90,$171.02,$2.90
30-34,37,$3.00,$111.15,$3.00
35-39,26,$3.29,$85.47,$3.29
40+,7,$3.08,$21.53,$3.08


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame

